# Importación de módulos

In [1]:
import os
import pandas as pd

# Lectura de la Base de Datos
Para descargar la base de datos original se necesita ir a la siguiente [ruta](https://sio.cnsf.gob.mx/)

## Definición de ruta

In [2]:
read_dir = r'C:\Users\ed_22\Documents\SIO'
read_file = 'ER_dic2023.xlsx'
read_path = os.path.join(read_dir, read_file)

## Variables globales

In [3]:
FIELDS = {
    'FECHA_CORTE':'corte',
    'NOMBRE_CORTO':'nombre',
    'DESCRIPCION':'cuenta',
    'DESC_OPERACION':'ramo',
    'IMPORTE':'importe'
}

# Lectura de base de datos

In [4]:
ER = pd.read_excel(read_path, usecols=FIELDS.keys()).rename(columns=FIELDS)

In [5]:
ER.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 536897 entries, 0 to 536896
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype         
---  ------   --------------   -----         
 0   corte    536897 non-null  datetime64[ns]
 1   nombre   536897 non-null  object        
 2   cuenta   536897 non-null  object        
 3   ramo     536897 non-null  object        
 4   importe  536897 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 20.5+ MB


In [6]:
ER.nombre.value_counts()

Seguros Atlas                  17608
Seguros Inbursa                14715
Mapfre México                  13518
Grupo Nacional Provincial      13504
AXA Seguros                    13409
                               ...  
Seguros de Vida Sura México     1084
AXA Salud                       1077
MBIA México                     1053
CRABI                            900
Principal Pensiones              678
Name: nombre, Length: 114, dtype: int64

In [7]:
ER.ramo.unique()

array(['Total', '*Fianzas', '**Fidelidad', '***Individual',
       '***Colectivo', '**Judicial o Penal', '***Penales',
       '***No penales', '***Amparan conductores de automóviles',
       '**Administrativo', '***Por Obra', '***Proveeduría', '***Fiscales',
       '***Arrendamiento', '***Otras de Administrativo', '**De crédito',
       '***Suministro', '***Compra Venta', '***Otras de Crédito',
       '***Financiera', '*Da?os', '**Garantía financiera',
       '*Accidentes y Enfermedades', '***Gastos Médicos', '***Salud',
       '***Accidentes Personales', '*Pensiones',
       '***IMSS Incapacidad Permanente RT',
       '***IMSS Muerte (riesgos de trabajo)',
       '***IMSS Invalidez (invalidez y vida)',
       '***IMSS Muerte (invalidez y vida)',
       '***IMSS Jubilación (retiro, cesantía y vejez)',
       '***ISSSTE Incapacidad Permanente RT',
       '***ISSSTE Muerte (riesgos de trabajo)',
       '***ISSSTE Invalidez (invalidez y vida)',
       '***ISSSTE Muerte (invalidez y vida)'

In [11]:
ER.cuenta.unique()

array(['Directo', 'Primas Cedidas', 'Primas De Retención',
       'Primas de Retención Devengadas', 'Costo Neto de Adquisición',
       'Comisiones a Agentes',
       'Costo Neto de Siniestralidad, Reclamaciones y Otras Obligaciones Pendientes de Cumplir',
       'Utilidad (Pérdida) Técnica', 'Gastos de Operación Netos',
       'Resultado Integral de Financiamiento',
       'Utilidad (Pérdida) antes de Impuestos a la Utilidad',
       'Utilidad  (Pérdida) del Ejercicio',
       'Utilidad (Pérdida) de la Operación', 'Tomado',
       'Compensaciones Adicionales a Agentes'], dtype=object)

In [14]:
df = ER.copy()

In [16]:
df.head()

,corte,nombre,cuenta,ramo,importe
0,2016-03-31,Crédito Afianzador,Directo,Total,17.68
1,2016-06-30,Crédito Afianzador,Directo,Total,19.30
2,2016-09-30,Crédito Afianzador,Directo,Total,21.39
3,2016-12-31,Crédito Afianzador,Directo,Total,24.54
4,2017-03-31,Crédito Afianzador,Directo,Total,17.77


In [17]:
df['nombre'] = df.nombre.str.replace('?', 'ñ', regex=True)
df['cuenta'] = df.cuenta.str.replace('?', 'ñ', regex=True)
df['ramo'] = df.ramo.str.replace('*', '', regex=True)
df['ramo'] = df.ramo.str.replace('?', 'ñ', regex=True)